In [1]:
# Linear Algebra
import numpy as np
from scipy.linalg import svd
from numpy.linalg import qr
import scipy.sparse.linalg.eigen.arpack as arp

from timeit import timeit

# Python Plotting'
from matplotlib import pyplot as plt

# Own modules
from DMRG_022_LP import * # here is the main part of the DMRG code

# we'll need them a lot
sxx = np.array([[0., 1.], [1., 0.]])
syy = np.array([[0., -1j], [1j, 0.]])
szz = np.array([[1., 0.], [0., -1.]])

In [2]:
class M_MPO(object):
    '''
    MPO for the total magnetization operator M = \\sum_i \\sigma_i^z
    '''
    def __init__(self,L):
        self.L = L
        self.d = 2

        self.sz = np.array([[1., 0.], [0., -1.]])
        self.sx = np.array([[0.,1.],[1.,0.]])
        self.id = np.eye(2)
        self.Ws = self.init_W()  # [?] in toycode they do this a bit strange
                                #and make the defintion self.W inside init_W, is this necessary or rather 'style?'
    
    def init_W(self):
        # Create the matrix of operators (2x2 matrices)
        # (virutal left, virtual right, physical out, physical in)
        # vL, vR, s, s'
        Ws = []
        # First vector
        w = np.zeros((1,2,self.d,self.d),dtype=np.complex)
        w[0,0] = self.id
        w[0,1] = self.sx
        Ws.append(w)
        # Ws[1:L-2]
        w = np.zeros((2,2,self.d,self.d),dtype=np.complex)
        w[0,0] = self.id
        w[0,1] = self.sx
        w[1,1] = self.id
        # create L-2 times the same matrix
        for i in range(1,self.L-1):
            Ws.append(w)
        # Last vector
        w = np.zeros((2,1,self.d,self.d),dtype=np.complex)
        w[0,0] = self.sx
        w[1,0] = self.id
        Ws.append(w)
        return Ws

def dmrg_m_MPO(L,J,g,chi_max=50,conf=1e-4,test=False,eps=-1,tol=0,maxiter=None):
    '''
    calculate mean magnetization m = <\\sum_i \\sigma_i^z>
    FROM MPO
    '''
    # Perform DMRG
    ## Initialize random MPS and MPO
    init_mps = random_MPS(L=L,d=2,chi_max=chi_max)
    mpo = Ising_MPO(L=L,d=2,h=g,J=J)
    dmrg = DMRG(init_mps,mpo,eps=eps,chi_max=chi_max,test=False,tol=tol,maxiter=maxiter)
    ## Left- and Right-sweep
    diff = 5.*conf # just to make it bigger than conf
    counter = 0
    while diff > conf:
        if test:
            print(counter,diff)
        counter += 1
        ### note: we could also simply use the current dmrg.e value to speed things up. This is just to give justification to the expect_mpo(mps,mpo) function
        dmrg.left_to_right() # appends the current e-values for each site
        e_in = dmrg.e[-1]
        dmrg.right_to_left()
        e_end = dmrg.e[-1]
        diff = abs(e_in - e_end)/abs(e_in)
        #if test and counter > 1:
        #    print('diff = ',diff)
        if counter > 5:
            print('Exceeded 5 epochs')
            break
    if test:
        print('g = ',g,' - epochs = ',counter-1,' - rel. diff = ',100*diff,'%')
        print('E_dmrg = ',dmrg.e[-1])
    # Calculate magnetization
    return expect_mpo(dmrg.MPS,M_MPO(L))/L

In [ ]:
gs = np.linspace(0.,2.,11)
mag_MPO_30 = np.array([dmrg_m_MPO(L=30,J=-1.,g=-1*g,chi_max=40,test=True,conf=1e-6,eps=-1) for g in gs])
mag_MPO_50 = np.array([dmrg_m_MPO(L=50,J=-1.,g=-1*g,chi_max=40,test=True,conf=1e-6,eps=-1) for g in gs])
mag_MPO_70 = np.array([dmrg_m_MPO(L=70,J=-1.,g=-1*g,chi_max=40,test=True,conf=1e-6,eps=-1) for g in gs])

0 4.9999999999999996e-06
g =  -0.0  - epochs =  0  - rel. diff =  8.943037881118471e-13 %
E_dmrg =  -28.999999999999844
0 4.9999999999999996e-06
g =  -0.2  - epochs =  0  - rel. diff =  6.058338298401714e-14 %
E_dmrg =  -29.320859151574325
0 4.9999999999999996e-06
g =  -0.4  - epochs =  0  - rel. diff =  1.1727365734691711e-14 %
E_dmrg =  -30.294217466851215
0 4.9999999999999996e-06
g =  -0.6000000000000001  - epochs =  0  - rel. diff =  4.427465709733018e-07 %
E_dmrg =  -31.95661630391031
0 4.9999999999999996e-06


In [ ]:
np.savez('data/dmrg_MPO_L.npz',mag_MPO_30 = mag_MPO_30,mag_MPO_50 = mag_MPO_50,mag_MPO_70 = mag_MPO_70)

In [ ]:
data=np.load('data/dmrg_MPO_L.npz')
mag_MPO_30,mag_MPO_50,mag_MPO_70 = data['mag_MPO_30'],data['mag_MPO_50'],data['mag_MPO_70']

In [ ]:
fig, axes = plt.subplots(nrows=1, ncols=1,figsize=(10,6))
### changed minus in MPO to check something
ax=axes
ax.plot(gs,mag_MPO_30,'-',label='L = 30')
ax.plot(gs,mag_MPO_50,'-',label='L = 50')
ax.plot(gs,mag_MPO_70,'-',label='L = 70')
ax.set(xlabel='g', ylabel='m',
       title='chi_max=40,conf=1e-6,eps=1e-3')
ax.legend()
#fig.savefig("mag_comp.png")